# Iris Recognition for Biometrics



## Background:
Normal human vision consists of a series of rapid eye movements called saccades, which exhibit ballistic and abrupt motion behavior. By tracking optokinetic motion with displayed stimulus patterns on a screen, a person's visual performance can be measured from their gaze. Eye trackers are used in visual system research, psychology, psycholinguistics, marketing, and even to detect drowsiness in drivers.

## The problem:
Capturing and processing biodata in the real world is noisy and tedious. Describe a DL /AI architecture, algorithm, framework, bioinformatics approach, protocol, method, or approach you would take to process and extract human gaze from a camera

## Proposed Solution: 
For this problem, I will be using the Daugman Algorithm so that I will be able to detect the iris of an eye.

In [3]:
## imports
import numpy as np
import cv2
import itertools
import math

In [ ]:
def daugman(grey_image, center, max_radius, min_radius, step):
    
    
    